# 결측치 조치 및 pipeline 구성

## 1.환경준비 

### (1) 라이브러리, 함수 불러오기 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler

### (2) 데이터 불러오기

In [ ]:
data = pd.read_csv('https://bit.ly/3FsgwkJ')
data.head()

In [ ]:
data.info()

* test set 분리하기

In [ ]:
data0, test = train_test_split(data, test_size = 20, random_state = 20)
data0.shape, test.shape

In [ ]:
test

## 2.데이터 전처리

### (1) 불필요한 데이터 처리
* 불필요한 정보를 제외시킵시다.
    * Unique value (식별자) 는 모델링에서 제외시킵니다.

In [ ]:
drop_cols = ['PassengerId','Ticket','Name']
data1 = data0.drop(drop_cols, axis = 1)
data1.head()

### (2) NaN 조치1

**1) 결측치 조회하기**
* 결측치를 조회해 봅시다.
* 어떻게 해결하는게 좋을까요? 해결 방법에 따라 적용 시점이 달라집니다.

In [ ]:
temp1 = pd.DataFrame(data1.isna().sum())
temp2 = pd.DataFrame(data1.isna().sum()/data1.shape[0])
temp3 = pd.concat([temp1, temp2], axis =1 )
temp3.columns = ['NaN 건수', 'NaN 비율']
temp3

* 시각화

In [ ]:
plt.figure(figsize = (8,5))
sns.heatmap(data0.isna())
plt.show()

* 어떻게 조치하는게 좋을까요?
    * 삭제 
        * 행 : 이 시점에 수행(필수)
        * 열 : 이 시점에 수행(권장), 가변수화 전까지 수행
    * 채우기
        * 단일값 : 범주 - 가변수화 전까지 수행, 숫자 - 스케일링 전까지
        * 추정값 : 스케일링 이후

* 어떻게 조치 방법에 따라 처리 시점이 달라집니다.
    * Embarked는 최빈값으로 **지금** 채우고
    * Age는 KNNImputer로 **가변수화 후에** 채우겠습니다.

* NaN 행 삭제를 결정한다면...
    * 운영에서 NaN이 들어오면 그 역시 버리겠다는 의미 입니다. 
        * 그래도 괜찮다면...
        * 그러나 괜찮은 상황은 별로 없을 겁니다.

**2) 삭제**

In [ ]:
# 열 삭제(이후에는 불필요한 데이터 처리와 함께 코드를 수행합니다.)
drop_cols = ['Cabin']
data1 = data1.drop(drop_cols, axis = 1)
data1.head()

**3) 채우기 : 단일값**

* SimpleImputer 

https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html

In [ ]:
# 범주형 변수 : 최빈값으로 채우기
simpute_cols = ['Embarked']

# imputer 선언
s_imputer = SimpleImputer(strategy = 'most_frequent')

# fitting 및 적용
data1[simpute_cols] = s_imputer.fit_transform(data1[simpute_cols])
data1.isna().sum()

**4) 실습**
* data1을 temp로 복사한 후에 temp['Age'] 에 대해서 평균으로 채워 봅시다.

In [ ]:
temp = data1.copy()

In [ ]:


# imputer 선언
temp_imputer = SimpleImputer(strategy =     )

# fitting 및 적용
      = temp_imputer.   ()
temp.isna().sum()

### (3) Feature Engineering
* family 변수를 추가하려고 합니다. 가족과 관련된 변수가 SibSp, Parch 입니다. 이 둘을 더하고 자기(1)까지 포함시켜서 가족 수 변수를 만듭시다.
* 그리고, SibSp, Parch 는 제거합니다.

In [ ]:
data1['Family'] = data1['SibSp'] + data1['Parch'] + 1
data1.drop(['SibSp', 'Parch'], axis = 1, inplace = True)
data1.head()

### (4) 가변수화

In [ ]:
cat = {'Sex':["female", "male"]
       , 'Embarked':["C", "Q", "S"]
       , 'Pclass':[1,2,3]}

for k, v in cat.items():
    data1[k] = pd.Categorical(data1[k], categories=v, ordered=False)

data1.info()

In [ ]:
data1 = pd.get_dummies(data1, columns =cat.keys(), drop_first = True)

### (5) 데이터 분할

* x, y 분할

In [ ]:
target = 'Survived'
x = data1.drop(target, axis = 1)
y = data1.loc[:, target]

* train, val 분할

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .3, random_state = 20)

### (6) 스케일링

* 스케일링이 분포를 변환시키는 것은 아닙니다.
    * MinMaxScaler
    * StandardScaler
* 스케일링에 따라 성능에 차이가 조금 날 수는 있지만, 성능을 향상시키는 일반적인 방법은 아닙니다.
* 스케일링을 데이터프레임 전체에 적용하고 나면 결과는 np.array로 나옵니다.
    * 그대로 모델링할 수도 있고, 다시 데이터프레임으로 변환해도 좋습니다.

* scaler 생성 및 train에 적용

In [ ]:
x_cols = list(x_train)
x_cols

In [ ]:
# 선언
scaler = MinMaxScaler()

# fitting & 적용
x_train_s = scaler.fit_transform(x_train)

# (옵션) 데이터프레임 변환
x_train_s = pd.DataFrame(x_train_s, columns = x_cols)

* val에 적용

In [ ]:
# validation 적용
x_val_s = scaler.transform(x_val)

# (옵션) 데이터프레임 변환
x_val_s = pd.DataFrame(x_val_s, columns = x_cols)

### (7) NaN 조치2

* KNNImputer
https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
# 선언하기
k_imputer = 

# fitting & 적용하기
x_train_s = k_imputer.  (  )
x_train_s = pd.DataFrame(x_train_s, columns = x_cols)

In [ ]:
x_train_s.isna().sum()

* validation set에 적용하기

In [ ]:
# validation 적용
x_val_s = k_imputer.   (  )
x_val_s = pd.DataFrame(x_val_s, columns = x_cols)

In [ ]:
x_val_s.isna().sum()

-----------------

## 3.모델링

### (1) 모델생성

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import *

In [ ]:
model = DecisionTreeClassifier(max_depth = 5)
model.fit(x_train_s, y_train)

### (2) 검증

In [ ]:
pred = model.predict(x_val_s)

In [ ]:
print(confusion_matrix(y_val, pred))
print(classification_report(y_val, pred))

## 4.실습 : Data Pipeline 정리
* 모델이 운영환경에 배포되었다고 가정합니다.
* 이제 pipeline을 구성해 봅시다.
* 운영에서 new data가 주어졌을 때, 어떤 절차로 파이프라인을 구성해야 할까요?

In [ ]:
# new data : test
test

### (1) 전처리 코드 정리
* 전처리 단계에서 수행한 코드들을 아래 하나의 셀에 다 붙여 넣어봅시다.
* 조회용 코드는 포함시키지 않습니다. 

#### 1) 코드 그대로 붙이기

In [ ]:
# (1)불필요한 변수 삭제



# (2) NaN 조치
# 열 삭제



# 최빈값으로 채우기




# (3) FE




# (4) 가변수화




# (5) 데이터분할



# (6) 스케일링



# (7) NaN 조치2 : KNNImputer




#### 2) 코드 정리
* 붙인 후에 새로운 데이터(x)가 들어와서 적용되는 과정을 머리로 그리면서 코드를 정리해 봅시다.
    * 동일한 작업 합치기
    * 새로운 데이터에 적용하는 부분만 필요. (train 데이터로 fitting 하는 부분은 제거)

In [ ]:
# 데이터와 상관 없이 미리 정의 되어야 할 코드



In [ ]:
# 새로운 데이터가 들어왔을 때 처리되는 절차 코드

# (1)불필요한 변수 삭제


# (2) NaN 조치


# (3) FE


# (4) 가변수화


# (6) 스케일링


# (7) NaN 조치2 : KNNImputer



#### 3) test 데이터로 확인하기
* 새로운 데이터란 운영환경에서 주여진 데이터라는 의미입니다.
* 그러므로 Label(target)이 없는 상태 입니다.
* test에서 Label을 제거한 후에 전처리 함수를 수행해 봅시다.

In [ ]:
temp0 = test.drop(target, axis =1)

# (1)불필요한 변수 삭제


# (2) NaN 조치


# (3) FE


# (4) 가변수화


# (6) 스케일링


# (7) NaN 조치2 : KNNImputer



### (2) 전처리 함수 생성 및 테스트

#### 1) 전처리 함수로 만들기
* 정리된 코드를 복사해 놓고 함수로 만들어 봅시다.
    * 입력으로 필요한 것은?
    * 무엇을 출력해야 할까요?

In [ ]:
def preprocessing(     ) :
    # input : data, s_imputer, cat, scaler, k_imputer
    # output : 전처리된 데이터셋

    # (1)불필요한 변수 삭제


    # (2) NaN 조치


    # (3) FE


    # (4) 가변수화


    # (6) 스케일링


    # (7) NaN 조치2 : KNNImputer


    return x_test

#### 2) 함수 적용
* x_test 만들기 : test에서 y(target) 제외하기

In [ ]:
test.head()

In [ ]:
temp0 = test.drop(target, axis =1)

x_test = preprocessing(    )
x_test.head()

#### 3) 예측
* x_test 로 예측결과를 뽑습니다.

In [ ]:
model.predict(x_test)

## 5.저장하기
* joblib을 이용하여 저장합니다.
* 새로운 세션(새로운 주피터노트북 파일)에서 새로운 데이터를 전처리 + 예측하고자 할 때 어떤 것들이 필요할까요?

In [ ]:
import joblib

### (1) 데이터

* test 데이터

In [ ]:
joblib.dump(test, 'test.pkl')

* x_column name

In [ ]:
# x column name
joblib.dump(x_cols, 'x_cols.pkl')

* categoriy 설정

In [ ]:
joblib.dump(cat, 'category_dict.pkl')

### (2) fitting된 함수들

In [ ]:
# s_imputer, scaler, k_imputer
joblib.dump(s_imputer, 's_imputer.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(k_imputer, 'k_imputer.pkl')

In [ ]:
# 모델
joblib.dump(model, 'model.pkl')

### (3) 함수
* 모듈로 저장하기(.py 파일)

In [ ]:
%%writefile preprocess.py

import pandas as pd
import numpy as np

def preprocessing(data, s_imputer, scaler, k_imputer, x_cols) :

    drop_cols = ['PassengerId','Ticket','Name', 'Cabin']
    data1 = data.drop(drop_cols, axis = 1)

    # 칼럼추가
    data1['Family'] = data1['SibSp'] + data1['Parch'] + 1
    data1.drop(['SibSp', 'Parch'], axis = 1, inplace = True)

    # NaN 조치 : 범주형 변수 최빈값으로 채우기
    s_impute_cols = ['Embarked']
    data1[s_impute_cols] = s_imputer.transform(data1[s_impute_cols])

    # 가변수화
    cat = {'Sex':["female", "male"]
        , 'Embarked':["C", "Q", "S"]
        , 'Pclass':[1,2,3]}

    for k, v in cat.items():
        data1[k] = pd.Categorical(data1[k], categories=v, ordered=False)

    data1 = pd.get_dummies(data1, columns =cat.keys(), drop_first = 1)

    # 스케일링
    data1_s = scaler.transform(data1)

    # NaN 조치
    data1_s = k_imputer.transform(data1_s)
    data1_s = pd.DataFrame(data1_s, columns = x_cols)
    
    return data1_s